In [ ]:
#bring gestures to images, used to train data
import cv2
import imutils
import numpy as np
import os


captureImage = cv2.VideoCapture(0)
bgSub = cv2.createBackgroundSubtractorMOG2()

def newDest(nameFolder):
    if not os.path.exists(nameFolder):
        os.mkdir(nameFolder)
        

# MAIN FUNCTION
#-----------------
def main(handGName):
    nFrames = 0
    nImage = 0
    nImagesMax = 1500
    initiateCamera = False

    newDest("EmoteMePls/gesturesCaptured/" + str(handGName))
    captureImage = cv2.VideoCapture(0)
    bgSub = cv2.createBackgroundSubtractorMOG2()
    
    top, right, bottom, left = 300, 50, 350, 350

    while(True):
        grabbed, frame = captureImage.read()
        frame = cv2.flip(frame, 1)
        
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        #gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)


        # clone the frame
        clone = frame.copy()

        # get the height and width of the frame
        #(height, width) = frame.shape[:2]

        #need to investigate this block
        altMasking = cv2.inRange(hsv, np.array([2, 50, 60]), np.array([25, 150, 255]))
        # get the ROI
        #roi = frame[top:bottom, right:left]
        roi = cv2.bitwise_and(frame, frame, mask=altMasking)

        # convert the roi to grayscale and blur it
        gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        setGray = cv2.GaussianBlur(gray, (7, 7), 0)
        
        
        setSquare = np.ones((7, 7), np.uint8)
        setDilation = cv2.dilate(setGray, setSquare, iterations=2)
        setOpening=cv2.morphologyEx(setDilation,cv2.MORPH_CLOSE,setSquare)

        grabbed, thresholded = cv2.threshold(setOpening, 30, 255, cv2.THRESH_BINARY)
        thresholded = thresholded[right:right + left, top:top + bottom]
        (cnts,_) = cv2.findContours(thresholded.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)#[1]
        #(cnts,_) = cv2.findContours(thresholded.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if len(cnts) > 0:
            cnt = max(cnts, key=cv2.contourArea)
            if cv2.contourArea(cnt) > 10000 and nFrames > 50:
                ptTop, ptRight, ptBottom, ptLeft = cv2.boundingRect(cnt)
                nImage += 1
                newImg = thresholded[ptRight:ptRight + ptLeft, ptTop:ptTop + ptBottom]
                if ptBottom > ptLeft:
                    newImg = cv2.copyMakeBorder(newImg, int((ptBottom - ptLeft) / 2), int((ptBottom - ptLeft) / 2), 0, 0,
                                                  cv2.BORDER_CONSTANT, (0, 0, 0))
                elif ptLeft > ptBottom:
                    newImg = cv2.copyMakeBorder(newImg, 0, 0, int((ptLeft - ptBottom) / 2), int((ptLeft - ptBottom) / 2),
                                                  cv2.BORDER_CONSTANT, (0, 0, 0))
                newImg = cv2.resize(newImg, (50, 50))
                cv2.putText(frame, "Working", (30, 60), cv2.FONT_HERSHEY_TRIPLEX, 2, (127, 255, 255))
                cv2.imwrite("EmoteMePls/gesturesCaptured/" + str(handGName) + "/" + str(nImage) + ".jpg", newImg)
        
        #x, y, w, h = 300, 50, 350, 350
        #top, right, bottom, left = 10, 350, 225, 590
        cv2.rectangle(frame, (top, right), (top + bottom, right + left), (0, 255, 0), 2)
        cv2.putText(frame, str(nImage), (100, 200), cv2.FONT_HERSHEY_TRIPLEX, 1.5, (127, 127, 255))
        cv2.imshow("Try and hold still, capturing", frame)
        cv2.imshow("b&w", thresholded)
        
        # observe the keypress by the user
        keypress = cv2.waitKey(1) & 0xFF
        if keypress == ord('c'):
            if initiateCamera == False:
                initiateCamera = True
            else:
                initiateCamera = False
                nFrames = 0
        if initiateCamera == True:
            nFrames += 1
        if nImage == nImagesMax:
            break
        if keypress == ord("x"):# or nImage == nImagesMax:
            camera.release()
            cv2.destroyAllWindows()
            break

        # to get the background, keep looking till a threshold is reached
        # so that our running average model gets calibrated
'''    run_avg(gray, aWeight)
        else:
            # segment the hand region
            hand = segment(gray)

            # check whether hand region is segmented
            if hand is not None:
                # if yes, unpack the thresholded image and
                # segmented region
                (thresholded, segmented) = hand
                (thresholded, blackAndWhiteImage) = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
                # draw the segmented region and display the frame
                cv2.drawContours(clone, [segmented + (right, top)], -1, (0, 0, 255))
                #cv2.imshow("Thesholded", thresholded)
                cv2.imshow('Black white image', blackAndWhiteImage)'''


handGName = input("Give the Hand Gesture a number\n")
main(handGName)